In [7]:
import pandas as pd
import numpy as np
import scipy.stats as st
import plotly.graph_objects as go
import plotly.express as px
import sympy
import warnings
from pynse import *
warnings.filterwarnings("ignore")
nse = Nse()

In [ ]:
pd.set_option("display.max_rows",150)

In [ ]:
def bs_call(price, strike,t_exp,vol):
    vol = vol/100
    t_exp = t_exp / 365
    d1= ((np.log(price/strike) + ( 0.1 + (vol**2)/2 )*t_exp)) / (vol * np.sqrt(t_exp))
    d2= (np.log(price/strike) + (0.1 - ((vol**2)/2))*t_exp) / (vol * np.sqrt(t_exp))
#     print(d2,d1-vol*np.sqrt(t_exp))
    return ((price* st.norm.cdf(d1)) - strike* np.exp(-0.1*t_exp)*st.norm.cdf(d2))

def bs_put(price, strike,t_exp,vol):
    vol = vol/100
    t_exp = t_exp / 365
    d1= ((np.log(price/strike) + ( 0.1 + (vol**2)/2 )*t_exp)) / (vol * np.sqrt(t_exp))
    d2= (np.log(price/strike) + (0.1 - ((vol**2)/2))*t_exp) / (vol * np.sqrt(t_exp))
#     print(d2,d1-vol*np.sqrt(t_exp))
    return ((-price* st.norm.cdf(-d1))+strike* np.exp(-0.1*t_exp)*st.norm.cdf(-d2))

def bs_call_delta(price,strike,t_exp,vol):
    t_exp = t_exp/356
    vol = vol/100
    d1= ((np.log(price/strike) + ( 0.1 + (vol**2)/2 )*t_exp)) / (vol * np.sqrt(t_exp))
    return st.norm.cdf(d1)

def bs_put_delta(price,strike,t_exp,vol):
    t_exp = t_exp/356
    vol = vol/100
    d1= ((np.log(price/strike) + ( 0.1 + (vol**2)/2 )*t_exp)) / (vol * np.sqrt(t_exp))
    return st.norm.cdf(d1) - 1 

def bs_call_theta(price,strike,t_exp,vol):
    vol = vol/100
    t_exp= t_exp / 365
    d1= float(((np.log(price/strike) + ( 0.1 + (vol**2)/2 )*t_exp)) / (vol * np.sqrt(t_exp)))
    d2= float((np.log(price/strike) + (0.1 - ((vol**2)/2))*t_exp) / (vol * np.sqrt(t_exp)))
    ndx = lambda x: (1/ np.sqrt(2*np.pi))  * (np.exp(- (x**2 /2)))
    return -(price * ndx(d1) * vol) / (2*np.sqrt(t_exp))  -  (0.1*strike*np.exp(-0.1*t_exp)*st.norm.cdf(d2))

def bs_put_theta(price,strike,t_exp,vol):
    vol = vol/100
    t_exp= t_exp / 365
    d1= ((np.log(price/strike) + ( 0.1 + (vol**2)/2 )*t_exp)) / (vol * np.sqrt(t_exp))
    d2= (np.log(price/strike) + (0.1 - ((vol**2)/2))*t_exp) / (vol * np.sqrt(t_exp))
    ndx = lambda x: (1/ np.sqrt(2*np.pi))  * (np.exp(- (x**2 /2)))
    return -(price * ndx(d1) * vol) / (2*np.sqrt(t_exp))  +  (0.1*strike*np.exp(-0.1*t_exp)*st.norm.cdf(d2))
def bs_gamma(price,strike,t_exp,vol):
    vol =vol/100
    t_exp = t_exp/365
    ndx = lambda x: (1/ np.sqrt(2*np.pi))  * (np.exp(- (x**2 /2)))
    d1= ((np.log(price/strike) + ( 0.1 + (vol**2)/2 )*t_exp)) / (vol * np.sqrt(t_exp))
    return ndx(d1) / (price*vol*np.sqrt(t_exp))
def bs_vega(price,strike,t_exp,vol):
    vol =vol/100
    t_exp = t_exp/365
    d1= ((np.log(price/strike) + ( 0.1 + (vol**2)/2 )*t_exp)) / (vol * np.sqrt(t_exp))
    ndx = lambda x: (1/ np.sqrt(2*np.pi))  * (np.exp(- (x**2 /2)))
    return price*np.sqrt(t_exp)*ndx(d1)
    
def inverter(z,l = [float(0.001),1]):
    if z(l[0]) == 0:
        return l[0]
    if z(l[1]) == 0:
        return l[1]
    if (l[1] -l[0]) < float (0.001):
#         print(*l)
        return  (float((l[0]+l[1]) /2))
    mid = float(l[0]+(l[1] - l[0])/2 )
    if z(mid) < 0 :
        l[0] = mid
        return float(inverter(z,l))
    if z(mid) > 0 :
        l[1] = mid
        return float(inverter(z,l))
    
def bs_call_iv (price,strike,t_exp,ce_price):
    t_exp = float(t_exp/356)
    vol_f = lambda vol: ((price* st.norm.cdf(float((np.log(price/strike) + ( 0.1 + ((vol**2)/2) )*t_exp)) / (vol * np.sqrt(t_exp)))) - strike* np.exp(-0.1*t_exp)*st.norm.cdf((np.log(price/strike) + (0.1 - ((vol**2)/2))*t_exp) / (vol * np.sqrt(t_exp)))) - (ce_price) 
    return inverter(vol_f) * 100

Testing above function

In [ ]:
bs_call(410,405,26,60)

In [ ]:
bs_call_iv(410,405,26,30.04050)

In [ ]:
nse.expiry_lisy()

In [ ]:
exp = nse.expiry_list[0]

In [ ]:
nse.option_chain("NIFTY",exp)

In [ ]:
# %%timeit

df = nse.option_chain("NIFTY",expiry= exp)

In [ ]:
df.columns = df.columns.str.lower().str.replace(".","_")

In [ ]:
arr_df = df[['pe_openinterest','pe_changeinopeninterest','pe_impliedvolatility','pe_lastprice','pe_change','pe_bidqty','pe_bidprice','pe_askprice',
   'pe_askqty','strikeprice','ce_bidqty', 'ce_bidprice','ce_askprice','ce_change','ce_lastprice','ce_impliedvolatility','ce_changeinopeninterest','ce_openinterest'
   ]]

In [ ]:
arr_df.columns

In [ ]:
fut_ltp = nse.get_quote("NIFTY", segment = Segment.FUT)["lastPrice"]

In [ ]:
arr_df = arr_df.loc[(arr_df['strikeprice']>(fut_ltp - fut_ltp*0.03)) & (arr_df['strikeprice']<(fut_ltp + fut_ltp*0.03)) ]
# days_exp = (nse.expiry_list[0] - dt.datetime.now().date()).days
print(fut_ltp)

In [ ]:
def syn_fut(inst,exp):
    op_chain = nse.option_chain(inst,exp)
    op_chain.index  = op_chain["strikePrice"]
    mon_fut  = nse.get_quote("NIFTY", segment = Segment.FUT)["lastPrice"]
    strk = round(mon_fut/50)*50
    return strk + op_chain["CE.lastPrice"][strk] - op_chain["PE.lastPrice"][strk] 

In [ ]:
syn_fut("NIFTY",dt.date(2022,6,2))

In [ ]:
t_exp = (exp - dt.datetime.now().date()).days
syn_futr = syn_fut("NIFTY",exp)

arr_df["Delta_call"] = arr_df.apply(lambda row: bs_call_delta(syn_futr,row["strikeprice"],t_exp,row['ce_impliedvolatility']), axis = 1)
arr_df["Delta_put"] = arr_df.apply(lambda row: bs_put_delta(syn_futr,row["strikeprice"],t_exp,row['pe_impliedvolatility']), axis = 1)

arr_df["Gamma_call"] =arr_df.apply(lambda row: bs_gamma(syn_futr,row["strikeprice"],t_exp,row['ce_impliedvolatility']), axis= 1)
arr_df["Gamma_put"] =arr_df.apply(lambda row: bs_gamma(syn_futr,row["strikeprice"],t_exp,row['pe_impliedvolatility']), axis= 1)

arr_df['Theta_call']=arr_df.apply(lambda row: bs_call_theta(syn_futr,row["strikeprice"],t_exp,row['ce_impliedvolatility']) / 365, axis= 1 )
arr_df['Theta_put']=arr_df.apply(lambda row: bs_put_theta(syn_futr,row["strikeprice"],t_exp,row['pe_impliedvolatility']) / 365, axis= 1 )

arr_df['Vega_call'] = arr_df.apply(lambda row:bs_vega(syn_futr,row['strikeprice'], t_exp,row['ce_impliedvolatility']), axis = 1)
arr_df['Vega_put'] = arr_df.apply(lambda row:bs_vega(syn_futr,row['strikeprice'], t_exp,row['pe_impliedvolatility']), axis = 1)

In [ ]:
req_columns = ['Delta_call','Gamma_call','Theta_call','Vega_call','ce_lastprice','ce_bidprice', 'ce_askprice',
    'strikeprice','pe_bidprice', 'pe_askprice','Delta_put','Gamma_put','Theta_put','Vega_put']

In [ ]:
arr_df[req_columns]